In [1]:

# Supress info messages
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sys
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

import pandas as pd
import time

print(tf.__version__)

2.13.0


In [2]:
url = 'https://raw.githubusercontent.com/mcstllns/DeepLearning/main/phone_price_classification_train.csv'
df = pd.read_csv(url)
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
# normaliza un conjunto de datos
def norm(x, st):
    return((x - st['mean'])/st['std'])
    
# Definimos la funcion para dibujar la historia

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  plt.plot(hist['epoch'], hist['loss'],'r--',
           label='Training Error')
  plt.plot(hist['epoch'], hist['val_loss'],'b',
           label = 'Validation Error')
  # plt.ylim([0,20])
  plt.legend()
  plt.show()

In [4]:
x_train = df.sample(frac=0.8, random_state=0)
x_test = df.drop(x_train.index)

y_train = x_train.pop('price_range')
y_test = x_test.pop('price_range')

In [5]:
y_train = to_categorical(y_train, num_classes=None)
y_test = to_categorical(y_test, num_classes=None)

In [6]:
x_train_stats = x_train.describe().transpose()
x_train_stats.head()

,count,mean,std,min,25%,50%,75%,max
battery_power,1600.0,1236.876250,441.779643,501.0,843.000,1225.0,1619.0,1998.0
blue,1600.0,0.497500,0.500150,0.0,0.000,0.0,1.0,1.0
clock_speed,1600.0,1.516000,0.822065,0.5,0.675,1.5,2.2,3.0
dual_sim,1600.0,0.511250,0.500030,0.0,0.000,1.0,1.0,1.0
fc,1600.0,4.355625,4.401564,0.0,1.000,3.0,7.0,19.0


In [7]:
x_test_stats = x_test.describe().transpose()
x_test_stats.head()


,count,mean,std,min,25%,50%,75%,max
battery_power,400.0,1245.08750,430.324293,503.0,879.75,1232.5,1595.0,1996.0
blue,400.0,0.48500,0.500401,0.0,0.00,0.0,1.0,1.0
clock_speed,400.0,1.54725,0.791808,0.5,0.70,1.5,2.2,3.0
dual_sim,400.0,0.50250,0.500620,0.0,0.00,1.0,1.0,1.0
fc,400.0,4.12500,4.092217,0.0,1.00,3.0,7.0,17.0


In [8]:
x_train_norm = norm(x_train, x_train_stats)
x_train_norm.head()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
405,0.491475,1.004698,-1.235912,0.977442,-0.762371,-1.041811,0.108651,0.690078,-1.608406,-0.220877,-1.138697,-0.905773,-0.517895,1.214527,-1.249717,-0.165111,-1.103008,0.562738,0.986035,-1.013529
1190,-0.327938,1.004698,-1.235912,0.977442,1.282357,-1.041811,-1.156252,-0.002380,0.752016,-0.657178,0.680407,-0.416606,-1.597744,-1.264267,0.407105,-0.395031,-0.006170,-1.775914,0.986035,-1.013529
1132,0.649925,1.004698,0.345471,0.977442,-0.989563,-1.041811,-1.211248,0.343849,0.948718,-0.220877,-1.469443,-1.121186,-1.646828,0.529670,0.880482,-0.165111,0.359443,0.562738,-1.013529,0.986035
731,1.290516,1.004698,0.710406,-1.022439,-0.535179,-1.041811,0.933588,1.036308,-0.428195,-1.529780,0.018915,-0.710555,0.302510,-0.183838,1.117171,1.674249,0.359443,-1.775914,0.986035,0.986035
1754,-0.341519,1.004698,0.223826,0.977442,-0.989563,0.959267,0.603613,-1.041068,-0.821599,0.651725,-1.469443,-1.341087,-0.244427,1.090680,-0.302962,-0.165111,1.090669,0.562738,0.986035,-1.013529


In [9]:
x_test_norm = norm(x_test, x_test_stats)
x_test_norm.head()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
11,0.638850,1.029175,0.824379,-1.003756,0.213821,0.95,0.048009,-0.020899,1.058324,1.505664,1.296537,-1.057048,-0.521119,1.485477,0.380729,0.692776,0.382036,0.545852,1.024038,0.983879
23,0.829404,1.029175,1.582139,0.993768,-0.030546,0.95,0.325520,0.675749,-0.758046,-0.693991,1.622300,-0.333027,-1.003806,-1.064733,-1.058248,0.236627,1.667735,0.545852,-0.974085,-1.013845
24,-1.724484,-0.969223,-0.438553,0.993768,0.213821,0.95,-1.339545,-0.369224,-0.844540,-0.693991,0.482129,-0.942124,0.012729,0.348901,-0.338759,-1.359893,0.198365,0.545852,-0.974085,-1.013845
25,-0.660171,1.029175,-0.185967,0.993768,-1.008011,0.95,1.380062,0.327425,-0.758046,1.505664,-1.146686,-0.735260,0.433134,0.530862,1.340047,0.692776,-0.719991,0.545852,1.024038,0.983879
28,0.483153,-0.969223,0.066620,0.993768,1.924385,0.95,1.102551,-0.717548,-1.277009,-1.133922,1.296537,-0.974302,0.159537,0.156880,-0.578589,-1.131819,-0.168978,0.545852,1.024038,0.983879


In [10]:
# comprobamos que lo hemos hecho bien

x_train_norm.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
battery_power,1600.0,-7.327472e-17,1.0,-1.665709,-0.891567,-0.026883,0.864965,1.722858
blue,1600.0,-2.331468e-17,1.0,-0.994701,-0.994701,-0.994701,1.004698,1.004698
clock_speed,1600.0,-4.440892e-17,1.0,-1.235912,-1.023033,-0.019463,0.832051,1.805209
dual_sim,1600.0,4.662937e-17,1.0,-1.022439,-1.022439,0.977442,0.977442,0.977442
fc,1600.0,3.108624e-17,1.0,-0.989563,-0.762371,-0.307987,0.600781,3.327085
four_g,1600.0,3.330669e-18,1.0,-1.041811,-1.041811,0.959267,0.959267,0.959267
int_memory,1600.0,4.884981e-17,1.0,-1.651214,-0.881273,-0.001341,0.878592,1.758525
m_dep,1600.0,-2.209344e-16,1.0,-1.387298,-1.041068,-0.002380,1.036308,1.728766
mobile_wt,1600.0,9.992007e-17,1.0,-1.692707,-0.877800,-0.034792,0.843342,1.679324
n_cores,1600.0,1.459943e-16,1.0,-1.529780,-0.657178,-0.220877,1.088026,1.524327


In [12]:
model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape=19*2000))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.summary()


TypeError: 'int' object is not iterable

In [13]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    # optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.1),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9),
    metrics = ['accuracy'])

In [14]:
history = model.fit(x_train, y_train, verbose=0)     

ValueError: in user code:

    File "/nix/store/yx2nwr2dab9jsa28v36zyscywixfplfv-python3-3.11.6-env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/nix/store/yx2nwr2dab9jsa28v36zyscywixfplfv-python3-3.11.6-env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/nix/store/yx2nwr2dab9jsa28v36zyscywixfplfv-python3-3.11.6-env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/nix/store/yx2nwr2dab9jsa28v36zyscywixfplfv-python3-3.11.6-env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/nix/store/yx2nwr2dab9jsa28v36zyscywixfplfv-python3-3.11.6-env/lib/python3.11/site-packages/ker